<a href="https://colab.research.google.com/github/tinana2k/CS-5542---Lab01/blob/main/week1_embeddings_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 5542 — Week 1 Lab
## From Data to Retrieval: GitHub → Colab → Hugging Face → Embeddings

**Learning Goals:**
- Use GitHub for collaborative analytics workflows
- Run notebooks in Google Colab
- Load datasets and models from Hugging Face Hub
- Build an embedding-based retrieval system (mini-RAG)


### GenAI Systems Context (Mini-RAG)
This lab implements a **mini Retrieval-Augmented Generation (RAG)** pipeline:
- A **Transformer encoder** produces semantic embeddings
- A **vector index (FAISS)** enables fast retrieval
- Retrieved context is what a downstream **LLM** would use for grounded generation


## Step 1 — Environment Setup
Install required libraries. This may take ~1 minute.


In [ ]:
!pip install -q transformers datasets sentence-transformers faiss-cpu

## Step 2 — Load Dataset & Model from Hugging Face Hub
We use a lightweight news dataset and a sentence embedding model.


TASK

1. Find out any /ag_news dataset from the huggingface.
2. Look for "Use this dataset" button on the left side --> Use huggingface library option.
3. Copy the entire code and paste in the empty cell and run it successfully.

In [ ]:
#paste your dataset code from huggingface here

from datasets import load_dataset

dataset = load_dataset("fancyzhx/ag_news")


In [ ]:
texts = dataset["train"].select(range(200))
print(f"Selected {len(texts)} examples from the 'train' split.")

In [ ]:
texts[1:6]

TASK

1. Find out the sentence-transformer -
all-MiniLM-L6-v2 from the huggingface website.
2. Look for "Use this model" button on the left side --> Use sentence-transformer library option.
3. Copy the first 2 lines of the code and paste in the empty cell and run it successfully.

In [ ]:
#paste your first 2 lines of the sentence-transformer library code from the hugginface to load the model

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

## Step 3 — Create Embeddings
These vectors represent semantic meaning and enable retrieval before generation.


In [ ]:
embeddings = model.encode(texts, show_progress_bar=True)
print('Embedding shape:', embeddings.shape)

## Step 4 — Build a Vector Index (FAISS)
This simulates the retrieval layer in RAG systems.


In [ ]:
import faiss
import numpy as np

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))
print('Index size:', index.ntotal)

## Step 5 — Retrieval Function
Search for documents related to a query.


In [ ]:
def search(query, k=3):
    q_emb = model.encode([query])
    distances, indices = index.search(np.array(q_emb), k)
    return [texts[int(i)] for i in indices[0]]

## Step 6 — Try It!


In [ ]:
query = "stock market and economy"
top_chunks = search(query, k=3)

print(f"Top 3 chunks retrieved for query: '{query}'\n")
for i, chunk in enumerate(top_chunks):
    print(f"{i+1}. {chunk['text']}\n")

query = "artificial intelligence in healthcare"
top_chunks = search(query, k=3)

print(f"Top 3 chunks retrieved for query: '{query}'\n")
for i, chunk in enumerate(top_chunks):
    print(f"{i+1}. {chunk['text']}\n")

query = "no money"
top_chunks = search(query, k=3)

print(f"Top 3 chunks retrieved for query: '{query}'\n")
for i, chunk in enumerate(top_chunks):
    print(f"{i+1}. {chunk['text']}\n")


## Reflection
In 1–2 sentences, explain how embeddings enable retrieval before generation in GenAI systems.


Embeddings encode both queries and documents into a shared vector space, allowing the system to retrieve the most semantically relevant information before generation. This retrieved context grounds the model’s response in external data, improving accuracy and reducing hallucinations.